# TP 1 : Synthèse de textures

## Auteurs :
- BURGER Loan
- PETIT Lucas

## Date :
- 13/11/2023

## Importations

In [31]:
import random
import time
import numpy as np
import cv2
from PIL import Image

## Fonctions d'optimisation

In [32]:
def updateBoundingBox(bbox, pixel):
    bboxList = list(bbox)
    if pixel[0] < bbox[0][0]:
        bboxList[0] = (pixel[0], bbox[0][1])
    if pixel[0] > bbox[0][1] - 1:
        bboxList[0] = (bbox[0][0], pixel[0] + 1)
    if pixel[1] < bbox[1][1]:
        bboxList[1] = (pixel[1] + 1, bbox[1][0])
    if pixel[1] > bbox[1][0] - 1:
        bboxList[1] = (bbox[1][1], pixel[1] + 1)

    bbox = tuple(bboxList)
    return bbox

In [33]:
def NumberOfNeighborsMask(number_of_neighbors_mask, half_patch_size):
    for i in range(number_of_neighbors_mask.shape[0]):
        for j in range(number_of_neighbors_mask.shape[1]):
            if number_of_neighbors_mask[i, j] == -1:
                for k in range(i - half_patch_size, i + half_patch_size + 1):
                    for l in range(j - half_patch_size, j + half_patch_size + 1):
                        if 0 <= k < number_of_neighbors_mask.shape[0] and 0 <= l < number_of_neighbors_mask.shape[1]:
                            if number_of_neighbors_mask[k, l] != -1:
                                number_of_neighbors_mask[k, l] += 1
    return number_of_neighbors_mask


def updateNumberOfNeighborsMask(number_of_neighbors_mask, pixel, half_patch_size):
    for i in range(pixel[0] - half_patch_size, pixel[0] + half_patch_size + 1):
        for j in range(pixel[1] - half_patch_size, pixel[1] + half_patch_size + 1):
            if 0 <= i < texture_size and 0 <= j < texture_size:
                if number_of_neighbors_mask[i, j] != -1:
                    number_of_neighbors_mask[i, j] += 1
    number_of_neighbors_mask[pixel[0], pixel[1]] = -1
    return number_of_neighbors_mask

## Fonctions de l'algorithme

In [34]:
def GetPixelWithMostColoredNeighbors(number_of_neighbors_mask, bbox):
    max_neighbors = 0
    max_neighbors_pixel = (-1, -1)
    for i in range(number_of_neighbors_mask.shape[0]):
        for j in range(number_of_neighbors_mask.shape[1]):
            if bbox[0][0] <= i <= bbox[0][1] - 1 and bbox[1][1] <= j <= bbox[1][0] - 1:
                continue
            if number_of_neighbors_mask[i, j] > max_neighbors:
                max_neighbors = number_of_neighbors_mask[i, j]
                max_neighbors_pixel = (i, j)
    return max_neighbors_pixel

In [35]:
def GetNeighborhoodWindow(pixel, image, half):
    # pas besoin non (max et min)? deja verif avec le if avant je crois  ########################################
    y_start, y_end = max(pixel[0] - half, 0), min(pixel[0] + half, image.shape[0])
    x_start, x_end = max(pixel[1] - half, 0), min(pixel[1] + half, image.shape[1])
    window = image[y_start:y_end, x_start:x_end, :]
    return window

In [36]:
def SSD(template, sample):
    return np.sum((template - sample) ** 2)


def SSD_Gaussian(template, sample):
    sigma = template.shape[0] / 6.4
    difference = template - sample
    ssd = np.sum((difference ** 2) * np.exp(-difference ** 2 / (2 * sigma ** 2)))
    return ssd


In [37]:
def FindMatches(template, sample_image, half_patch_size, epsilon):
    y_max, x_max, _ = sample_image.shape

    pixel_list = []
    min_ssd = float('inf')

    for y in range(half_patch_size, y_max - half_patch_size):
        for x in range(half_patch_size, x_max - half_patch_size):
            sample_patch = sample_image[y - half_patch_size:y + half_patch_size,
                           x - half_patch_size:x + half_patch_size, :]
            ssd_val = SSD(template, sample_patch)
            if ssd_val < min_ssd:
                min_ssd = ssd_val

    threshold = min_ssd * (1 + epsilon)

    for y in range(half_patch_size, y_max - half_patch_size):
        for x in range(half_patch_size, x_max - half_patch_size):
            sample_patch = sample_image[y - half_patch_size:y + half_patch_size,
                           x - half_patch_size:x + half_patch_size, :]
            if SSD(template, sample_patch) <= threshold:
                pixel_list.append((y, x))

    return pixel_list

## Fonction Utilitaire

In [38]:
def cropImage(image, size):
    y_max, x_max, _ = image.shape
    y_start = (y_max - size) // 2
    x_start = (x_max - size) // 2
    return image[y_start:y_start + size, x_start:x_start + size, :]

## Algorithme d'Efros et Leung

In [39]:
def efrosLeung(sample_image, texture_size, seed_size, half_patch_size, epsilon):
    # Choose a random seed
    half_seed_size = seed_size // 2
    seed_x = np.random.randint(0, sample_image.shape[1] - half_seed_size)
    seed_y = np.random.randint(0, sample_image.shape[0] - half_seed_size)

    # We want the seed to be in the center of the texture
    texture_center_x = texture_size // 2
    texture_center_y = texture_size // 2
    generated_texture = np.zeros((texture_size, texture_size, 3), dtype=np.uint8)
    generated_texture[texture_center_y - half_seed_size:texture_center_y + half_seed_size,
    texture_center_x - half_seed_size:texture_center_x + half_seed_size,
    :] = sample_image[seed_y - half_patch_size: seed_y + half_seed_size,
         seed_x - half_patch_size: seed_x + half_seed_size, :]

    # Initialize the number of neighbors mask
    number_of_neighbors_mask = np.zeros((texture_size, texture_size))
    number_of_neighbors_mask[texture_center_y - half_seed_size:texture_center_y + half_seed_size,
    texture_center_x - half_seed_size:texture_center_x + half_seed_size] = -1  # We don't want to update the number of neighbors of the seed

    # Initialize the bounding box
    bbox = [(texture_center_y - half_seed_size, texture_center_y + half_seed_size),
            (texture_center_x + half_seed_size, texture_center_x - half_seed_size)]

    # set the number of neighbors mask
    number_of_neighbors_mask = NumberOfNeighborsMask(number_of_neighbors_mask, half_patch_size)

    cv2.imshow('Generated Texture', generated_texture)
    # cv2.imshow("Bounding box", generated_texture[bbox[0][0]:bbox[0][1], bbox[1][1]:bbox[1][0], :])
    # cv2.imshow('Number of Neighbors Mask', number_of_neighbors_mask)
    cv2.waitKey(1)

    while (number_of_neighbors_mask != -1).any():
        pixel = GetPixelWithMostColoredNeighbors(number_of_neighbors_mask, bbox)
        bbox = updateBoundingBox(bbox, pixel)
        # Eviter les pixel trop proche des bordure de l'image de base pour ne pas avoir a resize l'image
        if pixel[0] < half_patch_size or pixel[0] > texture_size - half_patch_size or pixel[1] < half_patch_size or \
                pixel[
                    1] > texture_size - half_patch_size:  # We don't want to update the number of neighbors of the seed
            number_of_neighbors_mask = updateNumberOfNeighborsMask(number_of_neighbors_mask, pixel, half_patch_size)
            continue
        neighborhood_window = GetNeighborhoodWindow(pixel, generated_texture, half_patch_size)
        matches = FindMatches(neighborhood_window, sample_image, half_patch_size, epsilon)
        if matches == []:
            print("No match found for pixel ({}, {})".format(pixel[0], pixel[1]))
            break
        match = random.choice(matches)
        generated_texture[pixel[0], pixel[1], :] = sample_image[match[0], match[1], :]
        number_of_neighbors_mask = updateNumberOfNeighborsMask(number_of_neighbors_mask, pixel, half_patch_size)

        cv2.imshow('Generated Texture', generated_texture)
        # cv2.imshow('Number of Neighbors Mask', number_of_neighbors_mask)
        cv2.waitKey(1)  # Update display and wait for 1ms

    return generated_texture

## Main

In [40]:
start = time.time()

image_number = 0
texture_size = 70
seed_size = 24
epsilon = 0.01
half_patch_size = 12

sample_image = Image.open("textures_data/text{}.png".format(image_number))
np_sample_img = np.array(sample_image)

np_texture = efrosLeung(np_sample_img, texture_size, seed_size, half_patch_size, epsilon)

cropped_texture = cropImage(np_texture, texture_size - half_patch_size * 2)

texture = Image.fromarray(cropped_texture)

texture.save("generated_images/text{}_size{}_seed{}_patch{}_epsilon{}_GaussianSSD.png"
             .format(image_number,
                     texture_size,
                     seed_size,
                     half_patch_size,
                     epsilon)
             )
texture.show()

end = time.time()
print("Time: {:.2f} seconds".format(end - start))

Time: 75.50 seconds
